In [11]:
import numpy as np
import pandas as pd
import time
from functools import partial

from scipy.stats import randint, uniform
from sim_lib import simulation

pd.options.mode.chained_assignment = None


In [12]:
def eps_greedy(history: pd.DataFrame, eps: float):
    if uniform.rvs() < eps:
        n = history.shape[0]
        return history.index[randint.rvs(0, n)]

    ctr = history['clicks'] / (history['impressions'] + 10)
    n = np.argmax(ctr)
    return history.index[n]

policy = partial(eps_greedy, eps=0.06)

In [13]:
# seed for homework
seed = 18475
np.random.seed(seed=seed)

start = time.time()
output = simulation(policy, n=200000, seed=seed)
end = time.time()
end - start

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated


196.35531902313232

In [14]:
# baseline regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(1540.7609683932544, 0.007703804841966272, 184)

In [15]:
output['history']

,impressions,clicks,lifetime,p
153,18970.0,4199.0,18003.025431,0.220134
162,228.0,26.0,1537.166719,0.113780
172,163.0,32.0,19648.592394,0.219968
173,170.0,18.0,12771.474990,0.122694
180,26.0,3.0,4655.819793,0.020061
182,6.0,0.0,889.624649,0.004621
183,1.0,0.0,15187.163761,0.073886


## HW_3 Solution: 
### Будем использовать UCB 


In [23]:
def UCB(data_history, C, epsilon=1e-6):
    theta = data_history['clicks'] / (data_history['impressions'] + epsilon)
    t = np.sum(data_history["impressions"])
    exploration_term = C * (np.log(t+1) / (data_history['impressions'] + epsilon)) ** 0.5
    chosen_arm_index = np.argmax(theta + exploration_term)
    return data_history.index[chosen_arm_index]

In [24]:

eploration_param_list = [0.0001, 0.001, 0.01, 0.1, 1.0]


for C in eploration_param_list:
    
    # Define policy and run simulation
    policy = partial(UCB, C=C)
    seed = 18475
    np.random.seed(seed=seed)
    output = simulation(policy, n=200000, seed=seed)    
    end = time.time()
    end - start
    print("exploration parameter C:", C)
    print("regret:", output['regret'], "regret/rounds:", output['regret']/output['rounds'],  
          "total_banners:", output['total_banners'])
    


1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
exploration parameter C: 0.0001
regret: 4614.714232597847 regret/rounds: 0.023073571162989234 total_banners: 184
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impre

## При С == 0.1 regret значительно падает - рассмотрим ближние значения 

In [25]:
eploration_param_list = [0.03, 0.05, 0.07, 0.1, 0.3, 0.5, 0.7]


for C in eploration_param_list:
    
    # Define policy and run simulation
    policy = partial(UCB, C=C)
    seed = 18475
    np.random.seed(seed=seed)
    output = simulation(policy, n=200000, seed=seed)    
    end = time.time()
    end - start
    print("exploration parameter C:", C)
    print("regret:", output['regret'], "regret/rounds:", output['regret']/output['rounds'],  
          "total_banners:", output['total_banners'])
    

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
exploration parameter C: 0.03
regret: 1366.9655975486787 regret/rounds: 0.006834827987743394 total_banners: 184
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impres

# Results


1. Удалось побить бейзлайн 
UCB regret: 183.77075743210642
Baseline regret: 1540.7609683932544

2. Лучший С = 0.1